In [ ]:
# Standard library imports
import warnings
from pathlib import Path

# Third-party imports
import pandas as pd
import polars as pl

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

from settings import config
from pull_taq import get_taq_nbbo, get_taq_wct

In [ ]:
# Change default pandas display options

pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 200
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.expand_frame_repr', False)

# Global variables
RAW_DATA_DIR = Path(config("RAW_DATA_DIR"))
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WRDS_USERNAME = config("WRDS_USERNAME")

from pull_taq import get_taq_nbbo, get_taq_wct

In [ ]:
quotes = get_taq_nbbo(('SPY'), date='2024-03-07', use_bars=False, use_polars=True)
quotes

In [ ]:
trades = get_taq_wct(('SPY'), date='2024-03-07', use_polars=True)
trades